In [1]:
import os
import glob
import gc

import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader

from transformers import PatchTSTForPrediction

from datasets import Dataset

2025-11-04 11:11:28.973801: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data = "coin"

output_dir = "saved_models"
log_dir = os.path.join('logstf', data)

os.makedirs(log_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok = True)

loss_name = "MASE"

learning_rate = 5e-5
num_train_epochs = 400

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
## target domain
target_X = pd.read_csv(f"../data/{data}/train_input_7.csv").iloc[:, 1:].values.astype(np.float32)

np.random.seed(2)
random_indices1 = np.random.choice(pd.read_csv("../data/M4_train.csv").iloc[:, (1):].index,
                                   size=target_X.shape[0] * 20, replace=True)

X_data = pd.read_csv("../data/M4_train.csv").iloc[:, 1 + (24 * 0):].loc[random_indices1].values.astype(np.float32)
y_data = pd.read_csv("../data/M4_test.csv").iloc[:, 1:].loc[random_indices1].values.astype(np.float32)

## bootstrap
np.random.seed(42)
select = np.random.choice(len(X_data), size=len(X_data), replace=True)
X_bootstrap = X_data[select]
y_bootstrap = y_data[select]

val_split_index = int(len(X_bootstrap) * 0.8)

def to_tensor_and_reshape(array):
    result = torch.tensor(array)
    result = result.reshape(-1, result.shape[1], 1)

    return result

X_train, X_valid = to_tensor_and_reshape(X_bootstrap[:val_split_index]), to_tensor_and_reshape(X_bootstrap[val_split_index:])
y_train, y_valid = to_tensor_and_reshape(y_bootstrap[:val_split_index]), to_tensor_and_reshape(y_bootstrap[val_split_index:])

## setting dataloader
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 256, shuffle = True, num_workers = 16)

test_dataset = torch.utils.data.TensorDataset(X_valid, y_valid)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 256, num_workers = 16)

In [5]:
from tensorflow.keras.losses import Loss
import tensorflow as tf

class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)    ## 스케일 고정???

    def seasonal_diff(self, data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        diff = self.seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale

    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

In [6]:
y_train = y_train.numpy()
y_train = tf.convert_to_tensor(y_train)

I0000 00:00:1762222315.389742 2236009 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 55994 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 7g.80gb, pci bus id: 0000:47:00.0, compute capability: 8.0


In [7]:
y_train.shape, y_train[y_train.shape[1]:].shape

(TensorShape([11568, 24, 1]), TensorShape([11544, 24, 1]))

In [8]:
np.mean(np.abs(y_train[y_train.shape[1]:] - y_train[:-y_train.shape[1]]))

np.float32(3665.8032)

In [9]:
mase = MASE(y_train, y_train.shape[1])
mase.scale

np.float32(3665.8032)

> 해당 코드는 시점 간 차이가 아닌 서로 다른 시퀀스 간 차이를 계산함. 잘못된 구현.